In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 21291343872 (19.83 GB)
DFS Remaining: 21291294720 (19.83 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.3:9866 (project-4-lukee4-dn-1.project-4-lukee4_default)
Hostname: 9be8f6de71f9
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 15158603776 (14.12 GB)
DFS Remaining: 10645647360 (9.91 GB)
DFS Used%: 0.00%
DFS Remaining%: 41.23%
Configu

In [3]:
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2024-03-19 00:33:35--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv.11’

hdma-wi-2021.csv.11 100%[===================>] 166.84M   148MB/s    in 1.1s    

2024-03-19 00:33:36 (148 MB/s) - ‘hdma-wi-2021.csv.11’ saved [174944099/174944099]



In [4]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [5]:
!hdfs dfs -D dfs.replication=1 -Ddfs.blocksize=1048576 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.replication=2 -Ddfs.blocksize=1048576 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [6]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [7]:
#q3
!curl "http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"

{"FileStatus":{"accessTime":1710808423763,"blockSize":1048576,"childrenNum":0,"fileId":16386,"group":"supergroup","length":174944099,"modificationTime":1710808428369,"owner":"root","pathSuffix":"","permission":"644","replication":1,"storagePolicy":0,"type":"FILE"}}

In [8]:
#q4
#! curl "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()['Location']

'http://9be8f6de71f9:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [9]:
#q5
# might run into problem if first 2 blocks use same container id
r1 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
j1 = r1.json()['Location']
s1 = j1[7:19]

r2 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=1048576&noredirect=true")
j2 = r2.json()['Location']
s2 = j2[7:19]

count1 = 0
count2 = 0

for x in range(0, 174944099, 1048576):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=" + str(x) + "&noredirect=true")
    j = r.json()['Location']
    if(s1 in j):
        count1 += 1
    if(s2 in j):
        count2 += 1

Dict = {s1: count1, s2: count2}
Dict

{'9be8f6de71f9': 78, '3c093826594b': 89}

In [10]:
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

2024-03-19 00:34:05,462 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
#q6
f = hdfs.open_input_file("/single.csv")
f.read_at(10, 0)

b'activity_y'

In [12]:
#q7
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    count = 0
    for i, line in enumerate(reader):
        if("Single Family" in line):
            count += 1

count

444874